In [3]:
file = "/Users/paulie/Downloads/test_baroque.xlsx"

from openpyxl import load_workbook
wb = load_workbook(filename=file, read_only=True)

print(wb)
print(wb.worksheets)

[<ReadOnlyWorksheet "Plan">, <ReadOnlyWorksheet "fête">, <ReadOnlyWorksheet "dispositif">, <ReadOnlyWorksheet "perfomance">, <ReadOnlyWorksheet "expérience">, <ReadOnlyWorksheet "Fins">, <ReadOnlyWorksheet "a classer">, <ReadOnlyWorksheet "exclus">]


In [16]:
"""
Show all cells with some value
"""

for ws in wb.worksheets:
    for row in ws.rows:
        for cell in row:
            if cell.value is not None:
                print(cell.value)

FETE
DISPOSITIF
Ornement
Apparato
Général
Porte
Char 
Théâtre 
Fontaine
Grotte
Château
Pyramide
Obélisque
Colonne
Stade
Clôture
Objet lumineux
Luminaire
Pyrotechnie 
Objet sonore
Général
Artillerie 
Pétard 
Cloche
Instrument à cordes  
Instrument à vent 
Instrument à percussion 
Objet odorant
Objet de la représentation
Objet de procession
Objet militaires et de pouvoir
Objet liturgique
Général
Vaisselle sacrée
Parement d'autel
Vêtement et insignes écclésiastique
Procession et dévotion
Décor textile
Général
Objet textile
Tissu
Passementerie
Décor végétal 
Général
Motif, disposition
Espèce
Matériaux
Général
Or
Argent
Autre métal
Minéral, pierre 
Bois
Peinture, dessin, gravure
Végétal
Textile
Autre
Technique
Couleur
Général
Couleur
Représentation
image, figure
Inscription
Iconographie
Hagiographie
Mythologie 
Allégorie
Figure humaine, hybrides
Animal
Univers
Ornement
Motifs divers
Manière
PERFORMANCE
Acteurs
artiste
participant
spectateur
Actions
Religion
Cortège
Lumière et bruit
Exécutio

In [35]:
def read_block(data, emit_error=False):

    macro = None
    micro = None
    convention = None
    try:
        macro = data.pop(0).value
        micro = data.pop(0).value
        convention = data.pop(0).value
    except IndexError as e:
        if emit_error:
            raise IndexError("Could not read one of the block elements")
        return ()

    # Should skip every row which has not a value in the first 3 cells
    if macro is None and micro is None and convention is None:
        if emit_error:
            raise IndexError("All block elements are empty")
        return ()

    return (macro, micro, convention)


In [44]:
def get_sheet_data(ws):
    
    row_num = 0
    languages = []
    terms = {}
    last_language = "Unknown"
    
    for row in ws.rows:
        data = list(row)
        row_num += 1
        
        # Get the block
        block = read_block(data)
        if len(block) == 0:
            if row_num == 1:
                raise KeyError("Cannot find headers!")
            continue

        # Unpack the block, which are the first 3 elements removed from data
        macro, micro, convention = block

        cell_num = -1
        
        for element in data:
            cell_num += 1
            if element.value is None:
                if row_num == 1:
                    if last_language is None and element.value is not None:
                        raise KeyError("Cannot find language name")                    

                continue

            #print("TEST", cell_num, element.value)

            # First row (header) tells you which languages
            # Store languages names from cell 4 on
            if row_num == 1:
                last_language = element.value
                languages.append(last_language)
            else:
                if convention not in terms:
                    terms[convention] = {}
                terms[convention][languages[cell_num]] = element.value
        
        if row_num > 4:
            print("Languages", languages)
            print(terms)
            break


In [45]:
get_sheet_data(wb['dispositif'])

Languages ['Latin ', 'italien ', 'français']
{'décor, effet ornemental': {'italien ': 'adddobbo, addobbare, addobbato', 'Latin ': 'decus', 'français': 'decorer, decor'}, 'ornement < ordre': {'Latin ': 'ordo', 'français': 'appareil, appareilé, assorti'}, 'ornement': {'italien ': 'adornamento, ornamento, ornato', 'Latin ': 'ornatus, ornamentum, ad ornatus', 'français': 'ornement, orner'}}


for ws in wb.worksheets:
    get_sheet_data(ws)